In [1]:
#set up environment
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization

# create directories for the notebook. DO NOT change
data_dir = 'data/06_Kraken_results'
!data_dir = 'data/06_Kraken_results'

!mkdir -p data
!mkdir -p $data_dir

# fetches useful files for the current notebook. All files will be saved in $data_dir
!wget 'https://polybox.ethz.ch/index.php/s/BDifSidsoRrAaqX/download' -O data/Download.zip
!unzip -o data/Download.zip -d data
!rm data/Download.zip

/usr/bin/sh: line 1: data_dir: command not found
--2025-12-11 10:08:06--  https://polybox.ethz.ch/index.php/s/BDifSidsoRrAaqX/download
Resolving polybox.ethz.ch (polybox.ethz.ch)... 129.132.71.243
Connecting to polybox.ethz.ch (polybox.ethz.ch)|129.132.71.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘data/Download.zip’

data/Download.zip       [       <=>          ]  25.02M  19.1MB/s    in 1.3s    

2025-12-11 10:08:07 (19.1 MB/s) - ‘data/Download.zip’ saved [26236904]

Archive:  data/Download.zip
 extracting: data/06_Kraken_results/06_Interactive_script.sh  
 extracting: data/06_Kraken_results/06_Taxonomy_classification.slurms.sh  
 extracting: data/06_Kraken_results/Classify/kraken_hits_derep_50.qza  
 extracting: data/06_Kraken_results/Classify/kraken_reports_mags_derep_50.qza  
 extracting: data/06_Kraken_results/mags-taxonomy.qza  
 extracting: data/06_Kraken_results/taxonomy.qzv  
 extracting: data/06_Krake

# Taxonomy: Kraken2 based classification
For classification we decided to use the kraken2 Moshpit tool and used the combined dereplicated mags with pluspfp16 Kraken2 db. If you are not familiar with Kraken "pluspfp" contains Reference Sequences of the standard: archaea, bacteria, viral, plasmid, human1, UniVec_Core; in addition it also contains protozoa, fungi and plants. The rationale behind this choice is the attempt to classify the dereplicated mags that were extracted in step 03 from all 3 domains of life. A size-capped database was used due to limits in computational usage (16Gb). [^1]

### Step 1: downloading db

the following script was supposed to download and classify only bacteria dereplicates directly. It has not been modified since a an alternative was used for the actual classification (see next markdown) and the used database is the same

[^1] Wood et al. Genome Biology (2019) 20:257 https://doi.org/10.1186/s13059-019-1891-0


In [2]:
!head -n 35 $data_dir/06_Taxonomy_classification.slurms.sh

#!/bin/bash
#SBATCH --job-name=Kraken2_Taxonomy
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=20
#SBATCH --mem-per-cpu=4GB
#SBATCH --time=48:00:00
#SBATCH --output=/cluster/home/emotta/applied_bioinformatics/01_logs/%x_%j_out.log
#SBATCH --error=/cluster/home/emotta/applied_bioinformatics/01_logs/%x_%j_err.log

module load eth_proxy

# initialize conda manually
source /cluster/home/emotta/miniconda3/etc/profile.d/conda.sh
conda activate qiime2-moshpit-2025.7

#variables
CACHE_DIR=/cluster/scratch/emotta/cache
mkdir /cluster/scratch/emotta/03_artifacts/Kraken_DB
KRAKEN_DB=/cluster/scratch/emotta/03_artifacts/Kraken_DB
MAGS_DIR=/cluster/scratch/emotta/03_artifacts/dereplicates/dereplicates/mags_filtered_bacteria_dereplicates.qza

mkdir $KRAKEN_DB/Classify
mkdir $KRAKEN_DB/Taxonomy


# loop through dereplicated mags artifacts
 mosh annotate build-kraken-db \
    --p-collection pluspf \
    --p-threads 48 \
    --p-kmer-len 35 \
    --o-kraken2-db $CACHE_DIR:Kraken2_pluspf \
    --o-bracken-d

### Step 2: Classification
Premise: a bug in how moshpit handles cpus for this step did not allow to use a .slurms.sh. A pipeline of commands was fed to the interactive node instead.

As Previously mentioned the input is the combined of the 3 dereplicated mags obtained in Notebook 03. This command results in 2 files: the reports and the hits.

In [3]:
!cat $data_dir/06_Interactive_script.sh

#!/bin/bash
srun --pty -n 20 --mem-per-cpu=4G -t 08:00:00 bash #allocate resources to the interactive node 
conda activate qiime2-moshpit-2025.10 # activate conda
mosh annotate classify-kraken2 \ # Kraken classifier: uses dereplicated bacteria mags and a subset of pluspfp
    --i-seqs mags_filtered_bacteria_dereplicates.qza \
    --i-db /cluster/work/bokulich/DATABASES/QIIME2_CACHE:kraken2_pluspfp16 \ #chosen because fungi could be a good candidate for fermentation microbiome
    --p-threads 20 \
    --o-reports kraken_reports_mags_derep_50.qza \
    --o-outputs kraken_hits_derep_50.qza \
    --verbose

### Step 3: annotation
Annotation is necessary to combine the reports and the hits into a feature-table for the Taxonomy.
The end result is a visualization of the table of taxonomical classification of all dereplicated samples.

In [4]:
#annotate hits to tree
! qiime annotate kraken2-to-mag-features \
    --i-reports $data_dir/Classify/kraken_reports_mags_derep_50.qza \
    --i-outputs $data_dir/Classify/kraken_hits_derep_50.qza \
    --o-taxonomy $data_dir/mags-taxonomy.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureData[Taxonomy] to: data/06_Kraken_results/mags-taxonomy.qza


In [5]:
#create taxonomy viusalization
! qiime metadata tabulate \
    --m-input-file $data_dir/mags-taxonomy.qza \
    --o-visualization $data_dir/taxonomy.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: data/06_Kraken_results/taxonomy.qzv


In [6]:
Visualization.load(f"{data_dir}/taxonomy.qzv")

<visualization: Visualization uuid: 630d0425-4837-4906-9cd9-37f4d4097f6e>

### Step 4: Downstream analysis
this step is still a work in progress. the result we would like to achieve is the metacoder tree graph (seen in Week 11 lecture) to compare different country and food types with each other.

In [7]:
!qiime tools export \
    --input-path $data_dir/taxonomy.qzv \
    --output-path $data_dir/taxonomy_out

Exported data/06_Kraken_results/taxonomy.qzv as Visualization to directory data/06_Kraken_results/taxonomy_out


In [8]:
#setup R environment for metacoder plot

!pip install rpy2
%load_ext rpy2.ipython
import rpy2

In [9]:
%%R
a <- 1+1
print(a + 1)
#install.packages("metacoder")

[1] 3


In [10]:
%%R


In [13]:
%%R
install.packages("remotes")
remotes::install_github("grunwaldlab/metacoder")
library(metacoder)

# read table
tax <- read.table(
  "data/06_Kraken_results/taxonomy_out/metadata.tsv",
  header = TRUE,
  sep = "\t",
  stringsAsFactors = FALSE
)

# parse taxonomy. Replace column names accordingly:
dat <- parse_tax_data(
  tax,
  class_cols = "taxonomy",   # column with semicolon-separated taxonomy
  named_by_taxon_id = FALSE
)

# calculate per-taxon counts (replace "count" with your abundance column)
dat$data$tax_abund <- calc_taxon_abund(dat, "count")

# create heat tree
heat_tree(
  dat,
  node_size  = dat$data$tax_abund,
  node_color = dat$data$tax_abund
)


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/remotes_2.5.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 164496 bytes (160 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

These packages have more recent versions available.
It is recommended to update all of them.
Which would you like to update?

 1: All                                         
 2: CRAN packages only                          
 3: None                                        
 4: isoband      (0.2.7     -> 0.3.0    ) [CRAN]
 5: stringi      (1.8.4     -> 1.8.7    ) [CRAN]
 6: magrittr     (2.0.3     -> 2.0.4    ) [CRAN]
 7: xfun         (0.52      -> 0.54     ) [CRAN]
 8: xml2         (1.3.6     -> 1.5.1    ) [CRAN]
 9: stringr      (1.5.1     -> 1.6.0    ) [CRAN]
10: curl         (6.2.2     -> 7.0.0    ) [CRAN]
11: pillar       (1.11.0    -> 1.11.1   ) [CRAN]
12: hms          (1.1.3     -> 1.1.4    ) [CRAN]
13: RcppArmad... (14.4.2-1  -> 15.2.2-1 ) [CRAN]
14: pixmap       (0.4-13    -> 0.4-14   ) [CRAN]
15: igraph       (2.0.3     -> 2.2.1    ) [CRAN]
16: digest       (0.6.37    -> 0.6.39   ) [CRAN]
17: tinytex      (0.57      -> 0.58     ) [CRAN]
18: yaml         (2.3.10    -> 2.3.12   )

Enter one or more numbers, or an empty line to skip updates:  


R[write to console]: Installing 34 packages: S7, commonmark, litedown, markdown, quadprog, fastmatch, rex, ratelimitr, triebeard, pbapply, WikidataQueryServiceR, WikipediR, httpcode, urltools, crul, WikidataR, solrium, rncl, rentrez, gridtext, shades, segmented, phangorn, wikitaxa, natserv, worrms, ritis, rotl, rredlist, GA, cowplot, ggfittext, seqinr, taxize

R[write to console]: also installing the dependency ‘xfun’


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/xfun_0.54.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 169161 bytes (165 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write t

x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib    -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1714471492496/work=/usr/local/src/conda/r-base-4.3.3 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c base64.c -o base64.o
x86_64-conda-linux-gnu-cc -I"/opt/conda/lib/R/include" -DNDEBUG   -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib    -fpic  -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1714471492496/work=/usr/local/src/conda/r-base-4

...

* installing *source* package ‘metacoder’ ...
** using staged installation
** libs
using C++ compiler: ‘x86_64-conda-linux-gnu-c++ (conda-forge gcc 15.1.0-5) 15.1.0’


x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/Rcpp/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib    -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /opt/conda/include -fdebug-prefix-map=/home/conda/feedstock_root/build_artifacts/r-base-split_1714471492496/work=/usr/local/src/conda/r-base-4.3.3 -fdebug-prefix-map=/opt/conda=/usr/local/src/conda-prefix  -c RcppExports.cpp -o RcppExports.o
x86_64-conda-linux-gnu-c++ -std=gnu++17 -I"/opt/conda/lib/R/include" -DNDEBUG  -I'/opt/conda/lib/R/library/Rcpp/include' -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /opt/conda/include -I/opt/conda/include -Wl,-rpath-link,/opt/conda/lib    -fpic  -fvisibility-inlines-hidden  -fmessage-length=0 -march=nocona -mtune=haswell -ftree-vectorize -fP

installing to /opt/conda/lib/R/library/00LOCK-metacoder/00new/metacoder/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (metacoder)
R[write to console]: This is metacoder version 0.3.8 (stable)

R[write to console]: Error in parse_tax_data(tax, class_cols = "taxonomy", named_by_taxon_id = FALSE) : 
  unused argument (named_by_taxon_id = FALSE)




Error in parse_tax_data(tax, class_cols = "taxonomy", named_by_taxon_id = FALSE) : 
  unused argument (named_by_taxon_id = FALSE)


RInterpreterError: Failed to parse and evaluate line 'install.packages("remotes")\nremotes::install_github("grunwaldlab/metacoder")\nlibrary(metacoder)\n\n# read table\ntax <- read.table(\n  "data/06_Kraken_results/taxonomy_out/metadata.tsv",\n  header = TRUE,\n  sep = "\\t",\n  stringsAsFactors = FALSE\n)\n\n# parse taxonomy. Replace column names accordingly:\ndat <- parse_tax_data(\n  tax,\n  class_cols = "taxonomy",   # column with semicolon-separated taxonomy\n  named_by_taxon_id = FALSE\n)\n\n# calculate per-taxon counts (replace "count" with your abundance column)\ndat$data$tax_abund <- calc_taxon_abund(dat, "count")\n\n# create heat tree\nheat_tree(\n  dat,\n  node_size  = dat$data$tax_abund,\n  node_color = dat$data$tax_abund\n)\n'.
R error message: 'Error in parse_tax_data(tax, class_cols = "taxonomy", named_by_taxon_id = FALSE) : \n  unused argument (named_by_taxon_id = FALSE)'